This loads MGH LEADS data into redcap's MASTER project. Uses the RECON_FLAIR pipeline data.

In [91]:
import re
import getpass
import pandas as pd
import os
import pydicom
from pydicom.tag import Tag
import datetime
import glob
import math
import csv
from redcap import Project, RedcapError
import subprocess
import datetime
from ast import literal_eval
import numpy as np

api_url = 'https://redcap.partners.org/redcap/api/'
api_key = '49E6305CE4B41073E2342F723C0F93F8'
project = Project(api_url, api_key)

In [93]:
MGH_subjects = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/MGH_SUBJECTS.tsv', sep='\t', dtype={'ID': object})
#MGH_subjects = MGH_subjects.replace(np.nan, '', regex=True)
# need to convert all columns with numbers to int --> str
RECON_FLAIR_NOTES = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR/recon_notes.csv')
RECON_FLAIR_NOTES = RECON_FLAIR_NOTES.replace(np.nan, '', regex=True)


In [82]:
RECON_FLAIR_NOTES

,LEADS_ID,Recon_Quality,FS_QC_Notes,FLAG_FOR_IR,IR_TRIGGER,FS_INITIAL_SUBDIR,FS_INITIAL_COMMAND,STATUS_INITIAL,PBSJOB_INITIAL,FS_POSTEDIT_SUBDIR,FS_POSTEDIT_COMMAND,PBSJOB_EDIT,FS_VERSION,STATUS_FINAL,EDITOR,EXCLUDE,EXCLUDE_REASON,RECON_PATH
0,LDS0070120_20190620,Good,Pial edits mainly applied to cerebellum and la...,0,,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid LDS0070120_20190620 -FLAIR /...,,pbsjob_2562,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_2585,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
1,LDS0110021_20181016,Good,Some pial and artery edits.,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110021_20181016,complete,pbsjob_276,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_982,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
2,LDS0110022_20181129,Good,Some cerebellum edits and control points added...,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110022_20181129,complete,pbsjob_279,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -autorecon2-cp -...,pbsjob_2045,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
3,LDS0110040_20190123,Good,Some pial edits to temporal areas.,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110040_20190124,complete,pbsjob_311,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_2118,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
4,LDS0110041_20190212,Good,Many cerebellum edits and some pial edits.,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110041_20190215,complete,pbsjob_393,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_2065,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
5,LDS0110052_20190205,Good,Many pial surface corrections from the initial...,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110052_20190206,complete,pbsjob_385,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_2119,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
6,LDS0110053_20190227,Good,Corrected some poor pial surfaces.,0,,/autofs/cluster/animal/scan_data/leads/recon,recon-all -all -subjid LDS0110053_20190228,complete,pbsjob_384,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_2067,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
7,LDS0110054_20190522,Bad-Rerunning with BV,Grossly overestimated everywhere. Extremely ba...,1,Unusable recon. Thickness measures will be ove...,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid FS6_02_noflair -bigventricle...,complete,pbsjob_2122,,,,6,,RJE,,,
8,LDS0110078_20190320,Good,Added many control points to WM areas in tempo...,0,,/autofs/cluster/animal/scan_data/leads/recon_nip,recon-all -all -subjid LDS0110078_20190320,complete,pbsjob_725,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_02 -FLAIR /autofs/c...,pbsjob_1265,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...
9,LDS0110079_20190411,Good,A lot of white matter edits to frontal and tem...,0,,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid LDS0110079_20190411 -FLAIR /...,complete,pbsjob_1010,/autofs/cluster/animal/scan_data/leads/recon_n...,recon-all -subjid edit.FS6_05 -FLAIR /autofs/c...,pbsjob_1014,6,complete,RJE,0,,/autofs/cluster/animal/scan_data/leads/recon_n...


In [107]:
MGH_subjects

,dickerson_sessionid,leads_sessionid,session_completed,XNAT_upload,LONI_upload,MPRAGE_dropbox,MASTER_Updated,XNAT_comments_updated,headcoil,resolution,scanners,notes
0,CMID_F_63_20190123,LDS3600032_20190123,1,1,1,1,1,1,64ch,high,"TWD, RJE, RE",Possibly fell asleep during axial fcMRI/MBfMRI...
1,GGRI_M_66_20190320,LDS3600067_20190320,1,1,1,1,1,1,64ch,high,"KG, RJE, EK",Axial 3D pCASL some cerebellum cut-off.
2,PCER_M_58_20190118,LDS3600043_20190118,1,1,1,1,1,1,64ch,high,"RF, KG",Two MPRAGEs run but first was deleted during a...
3,PCIO_M_55_20181219,LDS3600030_20181219,1,1,1,1,1,1,64ch,high,"KG, RF",no comments
4,SGUR_F_62_20190323,LDS3600056_20190323,1,1,1,1,1,1,64ch,high,"RF, TD",Session: Second MPRAGE done due to contrast is...
5,JBRA_M_55_20190325,LDS3600068_20190325,2,1,1,1,1,1,64ch,high,"RF, EK",Slight movement in rsfMRI run. Hippocampus sca...
6,LSWA_F_58_20190417,LDS3600087_20190417,1,1,1,1,1,1,64ch,high,"RF, RJE",no comments
7,POCA_M_63_20190426,LDS3600069_20190426,2,1,1,1,1,1,64ch,high,"EK, RE, RJE",Significant amount of motion in all scans. Ran...
8,MMAH_M_60_20190430,LDS3600093_20190430,1,1,1,1,1,1,20ch,high,"RJE, RF",20-channel head coil used due to large head si...
9,RFOU_F_55_20190625,LDS3600105_20190625,1,1,1,1,1,1,64ch,high,"RJE, RE, MJB, RF",no comments


In [110]:
dry=0
saved_dataframes = []
# put all subjects and add to the dtaframe so that all info is imported!
sublist = ['SMAT_F_64_20190806'] #['POCA_M_63_20190426'] # ['MMAH_M_60_20190430'] # ['LSWA_F_58_20190417','PCER_M_58_20190118','PCIO_M_55_20181219','SGUR_F_62_20190323','MMAH_M_60_20190430','CMID_F_63_20190123','GGRI_M_66_20190320','POCA_M_63_20190426'] #,
for sub in sublist:
    
    name = sub.split("_")[0]
    date = sub.split("_")[3]
    date_sep = date[0:4]+'-'+date[4:6]+'-'+date[6:10]
    
    #obtain leads ID
    subject_init = sub.split('_')
    subject = subject_init[0]+'_'+subject_init[1]+'_'+subject_init[2]
    try:
        data_frame = project.export_records(records=[subject],forms=['identification_numbers'],events=[('static_arm_1')],format='df')
        saved_dataframes.append(data_frame)
        try:
            leads_id = 'LDS'+str(int(data_frame['leads_id'].values[0]))+'_'+date
        except(ValueError):
            leads_id = data_frame['leads_id'].values[0]+'_'+date
        print(leads_id)

        # find the redcap arm according to the date of the scan ## MAKE SURE TO ALSO ADD PROJECT CODE IN CASE TWO PROJECTS ARE COLLECTED ON ONE DAY
        session_of_interest = project.export_records(records=[subject],forms=['scan_session'],format='df')
        session_new = session_of_interest.reset_index()
        find_arm = session_new[(session_new['scan_date'] == date_sep) & (session_new['scan_irb_protocol'] == '2018P000859')] # must match IRB and scan date.
        arm = find_arm.loc[: , "redcap_event_name"].values[0]
        print(arm)

        # export scan_session form content of specified arm
        session_edit = project.export_records(records=[subject],forms=['scan_session'],events=[(arm)],format='df')
        #session_edit = session_edit.reset_index()

         #'1' for yes '' for nothing, '3' for incomplete, '2' for partial
        external_upload_complete = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].LONI_upload.values[0]
        session_edit.loc[:,'external_upload_complete'] = str(external_upload_complete)
        
        # string of 'RJE uploaded to LONI on MM-DD-YY' (NEED TO SPECIFICALLY ADD THIS COL TO DATAFRAME)
        specify_external_site = 'RJE uploaded to LONI'
        session_edit.loc[:,'specify_external_site'] = specify_external_site
        
        # names of scanners as strings
        scan_scanners = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].scanners.values[0]
        session_edit.loc[:,'scan_scanners'] = scan_scanners

        # complete, partial, incomplete
        scan_completion = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].session_completed.values[0]
        session_edit.loc[:,'scan_completion'] = str(scan_completion)
        
        # 8ch, 8-channel (MR-PET bays) | 12ch, 12-channel | 20ch, 20-channel head/neck | 32ch, 32-channel | 64ch, 64-channel
        scan_head_coil = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].headcoil.values[0]
        session_edit.loc[:,'scan_head_coil'] = scan_head_coil
        
        #high, high res structural | low, low res structural
        scan_mr_resolution = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].resolution.values[0]
        session_edit.loc[:,'scan_mr_resolution'] = scan_mr_resolution


        if MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].XNAT_upload.values[0] == 0:
            scan_uploaded_to_xnat = '0'
            print('WARNING: scan not on xnat')
        else:
            scan_uploaded_to_xnat = '1'
        session_edit.loc[:,'scan_uploaded_to_xnat'] = scan_uploaded_to_xnat
        scan_session_series_notes = MGH_subjects[MGH_subjects['dickerson_sessionid']==sub].notes.values[0]
        session_edit.loc[:,'scan_session_series_notes'] = scan_session_series_notes
        ###################

        cmd = [ 'findsession', name]
        output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
        trials = [f for f in str(output).split("=======") if (("SUBJECT") in f)]

        for trial in trials:
            m = re.search('DATE   :  (.+?)TIME', trial)
            found = m.group(1).replace(",","").strip('\\n')
            date_time_obj = datetime.datetime.strptime(found, '%B %d %Y')
            date_redcapformat = str(date_time_obj.date())
            date_conv = date_redcapformat.replace("-","")
            if date_conv == date:
                dpath = trial.split(':')[-1][:-1].strip('\\n').replace(" ","")
                break
            else:
                pass

        # now set all variables that I will import
        scan_upload_path = dpath 
        session_edit.loc[:,'scan_upload_path'] = scan_upload_path

        try:
            #yes / no
            scan_flag_for_review  = str(int((RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FLAG_FOR_IR).values[0]))
            session_edit.loc[:,'scan_flag_for_review'] = scan_flag_for_review

            #notes
            scan_flag_trigger = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].IR_TRIGGER).values[0]
            session_edit.loc[:,'scan_flag_trigger'] = scan_flag_trigger

            # text (string)
            recon1_qc_path = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].RECON_PATH).values[0]
            session_edit.loc[:,'recon1_qc_path'] = recon1_qc_path

            # read in RECON_FLAIR notes
            # 5_3_0, v5.3 | 6_beta, v6 beta | 6_0_0, v6.0 | dev, dev
            FSV = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FS_VERSION).values[0]
            if float(FSV) == 6:
                recon1_fsversion = '6_0_0'
            else:
                recon1_fsversion = ''
            session_edit.loc[:,'recon1_fsversion'] = recon1_fsversion

            # 1, good | 2, okay | 3, bad | 4, unknown | 5, flag
            recon_qc = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].Recon_Quality).values[0]
            if recon_qc == 'Good':
                recon1_quality = '1'
            elif recon_qc == 'Okay':
                recon1_quality = '2'
            elif recon_qc == 'Bad':
                recon1_quality = '3'
            elif (recon_qc == 'Unknown') or (recon_qc == ''):
                recon1_quality = '4'
            elif recon_qc == 'Flag':
                recon1_quality = '5'
            session_edit.loc[:,'recon1_quality'] = recon1_quality

            # yes, yes | inprog, in progress | flag_needs_xopts, flag - needs troubleshooting/xopts | no, no | exclude, exclude from processing (ie. no highres T1)
            recon_status = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].STATUS_FINAL).values[0]
            if recon_status == 'complete':
                recon1_qc_status = 'yes'
            elif recon_status == 'error':
                recon1_qc_status = 'flag_needs_xopts'
            else:
                recon1_qc_status = 'inprog'
            session_edit.loc[:,'recon1_qc_status'] = recon1_qc_status

            recon1_username = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].EDITOR).values[0]
            session_edit.loc[:,'recon1_username'] = recon1_username

            recon1_qc_notes = (RECON_FLAIR_NOTES[RECON_FLAIR_NOTES['LEADS_ID']==leads_id].FS_QC_Notes).values[0]
            session_edit.loc[:,'recon1_qc_notes'] = recon1_qc_notes

        except(IndexError):
            print('Subject '+sub+' is not found in the RECON_FLAIR_NOTES file. Please check that this subject has been run through nipype. Imported all non-QC info to RedCap.')
            pass
        except(ValueError):
            pass
#             session_edit.loc[:,'scan_flag_for_review'] = ''
#             session_edit.loc[:,'scan_flag_trigger'] = ''
#             session_edit.loc[:,'recon1_qc_path'] = ''
            
        #import with whatever info is available
        if dry==0:
            response = project.import_records(session_edit, date_format='YMD',format='df') # overwrite='overwrite'
            print('REDCAP UPDATED')
            # change the value of Redcap updated:
            MGH_subjects.loc[MGH_subjects['dickerson_sessionid']==sub, 'MASTER_Updated'] = '1'
            # float used to give errors in LEADS1! Make sure this doesn't happen again
            #MGH_subjects = MGH_subjects.astype(int)
            MGH_subjects.to_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/IDENTIFICATION/MGH_SUBJECTS.tsv', sep='\t', index=False)
        else:
            print('Dry run. Nothing has been loaded to redcap. Check variable session_edit')

        # add a status checker that redcap fields have been not been filled, or partially filled or completely filled
        if len(str(recon1_qc_path)) <5:
            print('recon is not available for sub '+sub)
            
    except(EmptyDataError):
        print('This subject '+sub+' does not exist in redcap.')

LDS3600133_20190806
visit_2_arm_1
REDCAP UPDATED


In [103]:
pd.set_option('display.max_columns', 500)
session_edit

,,scan_age_calc,calc_scan_age_years,calc_scan_age_mon_remaind,scan_date,scan_time_start,scan_time_end,scan_source,scan_project_id,scan_type,scan_pet_tracer,mr_event,scanner_location,outside_hospital_notes,mgh_scanner,cny_scanner_bay,scanner_model,scanner_software,scan_project_fund,scan_irb_protocol,scan_proj_timepoint,flag_for_external_upload,scan_priority,scan_notes,external_upload_complete,specify_external_site,scan_scanners,scan_completion,scan_head_coil,scan_mr_resolution,scan_pet_dose,scan_pet_mass,scan_uploaded_to_xnat,scan_session_series_notes,scan_upload_path,scan_accession_number,scan_bourget_andor_report,scan_flag_for_review,scan_flag_trigger,recon1_qc_path,recon1_fsversion,recon1_quality,recon1_qc_status,recon1_username,recon1_qc_notes,recon2_qc_path,recon2_fsversion,recon2_quality,recon2_qc_status,recon2_username,recon2_qc_notes,recon3_qc_path,recon3_fsversion,recon3_quality,recon3_qc_status,recon3_username,recon3_qc_notes,scan_amyloid_flr_visread,scan_petsurfer_flr,scan_amyloid_pos_neg,scan_status_petproc,scan_pet_qc_status,scan_pet_qc_notes,scan_pet_qcnotes,scan_petprocqc_who,petproc_procdir,petproc_fsdir,petproc_mrdd,petproc_allset,scan_findings_report,scan_session_complete
subject_id,redcap_event_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SMAT_F_64,visit_2_arm_1,55.275605,55,0.3,2019-08-06,13:00,15:00,research,LEADS,MR,NaN,NaN,CNY,NaN,NaN,CNY_bay4,NaN,NaN,1200_233636,2018P000859,bl,1,2,Upload to LONI,1.0,RJE uploaded to LONI,"MJB,SH,RJE",1.0,64ch,high,NaN,NaN,0,no comments,/cluster/archive/326/siemens/Prisma_fit-67026-...,NaN,NaN,0,,/autofs/cluster/animal/scan_data/leads/recon_n...,6_0_0,1,yes,RJE,Added pial edits to parietal areas and medial ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [109]:
type(scan_completion)

numpy.int64

In [75]:
MGH_subjects

,dickerson_sessionid,leads_sessionid,session_completed,XNAT_upload,LONI_upload,MPRAGE_dropbox,MASTER_Updated,XNAT_comments_updated,headcoil,resolution,scanners,notes
0,CMID_F_63_20190123,LDS3600032_20190123,1,1,1,1,1,1,64ch,high,"TWD, RJE, RE",Possibly fell asleep during axial fcMRI/MBfMRI...
1,GGRI_M_66_20190320,LDS3600067_20190320,1,1,1,1,1,1,64ch,high,"KG, RJE, EK",Axial 3D pCASL some cerebellum cut-off.
2,PCER_M_58_20190118,LDS3600043_20190118,1,1,1,1,1,1,64ch,high,"RF, KG",Two MPRAGEs run but first was deleted during a...
3,PCIO_M_55_20181219,LDS3600030_20181219,1,1,1,1,1,1,64ch,high,"KG, RF",no comments
4,SGUR_F_62_20190323,LDS3600056_20190323,1,1,1,1,1,1,64ch,high,"RF, TD",Session: Second MPRAGE done due to contrast is...
5,JBRA_M_55_20190325,LDS3600068_20190325,2,1,1,1,1,1,64ch,high,"RF, EK",Slight movement in rsfMRI run. Hippocampus sca...
6,LSWA_F_58_20190417,LDS3600087_20190417,1,1,1,1,1,1,64ch,high,"RF, RJE",no comments
7,POCA_M_63_20190426,LDS3600069_20190426,2,1,1,1,1,1,64ch,high,"EK, RE, RJE",Significant amount of motion in all scans. Ran...
8,MMAH_M_60_20190430,LDS3600093_20190430,1,1,1,1,1,1,20ch,high,"RJE, RF",20-channel head coil used due to large head si...
9,RFOU_F_55_20190625,LDS3600105_20190625,1,1,1,1,1,1,64ch,high,"RJE, RE, MJB, RF",no comments


In [76]:
MGH_subjects.astype(int)

ValueError: invalid literal for int() with base 10: 'CMID_F_63_20190123'

In [24]:
session_edit['recon1_qc_path'].values[0]

'/autofs/cluster/animal/scan_data/leads/recon_nip/RECON_FLAIR/LDS3600095_20190718/edit.FS6_02'